# Load data

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# Load dataframe
df = pd.read_csv('../data/OnlineNewsPopularity.csv', header=0, sep=', ', engine='python')

# Drop columns
df.drop(columns=['url', 'timedelta'], inplace=True)

# Binarize target column
df['shares'] = df['shares'].values > 1400

# Split into X, Y
values = df.values.astype(float)
X_cols = np.array(df.columns) != 'shares'
X, Y = values[:, X_cols], values[:, ~X_cols]
feature_names = np.array(df.columns)[X_cols]
num_features = X.shape[1]

# Train/val/test split
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.1, random_state=123)
X_train, X_val, Y_train, Y_val = train_test_split(
    X_train, Y_train, test_size=0.1, random_state=123)

In [3]:
# Standardize continuous columns
ss = StandardScaler()
ss.fit(X_train)
X_train = ss.transform(X_train)
X_val = ss.transform(X_val)
X_test = ss.transform(X_test)

# Set up imputer

In [4]:
import torch
import fastshap_torch
from fastshap_torch import Surrogate

In [5]:
device = torch.device('cuda', 4)
surrogate = torch.load('../models/news_surrogate.pt').eval().to(device)
num_features = X_train.shape[1]
surr = Surrogate(surrogate, num_features)

# FastSHAP

In [6]:
import torch
import torch.nn as nn
from fastshap_torch import FastSHAP

# Test samples number

In [7]:
for n_samples in (1, 4, 16, 32, 48, 64, 96):
    # Set up explainer model
    explainer = nn.Sequential(
        nn.Linear(num_features, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, 2 * num_features)).to(device)

    # Set up FastSHAP wrapper
    fastshap = FastSHAP(explainer, surr, normalization='additive', link=nn.Softmax(dim=1))

    # Train
    fastshap.train(
        X_train,
        X_val[:500],
        batch_size=32,
        num_samples=n_samples,
        max_epochs=200,
        eff_lambda=0,
        paired_sampling=False,
        validation_samples=128,
        validation_seed=123,
        lookback=5,
        verbose=False)

    # Print performance
    print('Best val loss = {:.8f}'.format(min(fastshap.loss_list)))

    # Save model
    modifier = 'samples={}'.format(n_samples)
    explainer.cpu()
    torch.save(explainer, '../models/news_explainer {} nopenalty.pt'.format(modifier))

Best val loss = 0.01873461
Best val loss = 0.00275944
Best val loss = 0.00032966
Best val loss = 0.00032586
Best val loss = 0.00032560
Best val loss = 0.00032520
Best val loss = 0.00032291


In [8]:
for n_samples in (4, 16, 32, 48, 64, 96):
    # Set up explainer model
    explainer = nn.Sequential(
        nn.Linear(num_features, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, 2 * num_features)).to(device)

    # Set up FastSHAP wrapper
    fastshap = FastSHAP(explainer, surr, normalization='additive', link=nn.Softmax(dim=1))

    # Train
    fastshap.train(
        X_train,
        X_val[:500],
        batch_size=32,
        num_samples=n_samples,
        max_epochs=200,
        eff_lambda=0,
        paired_sampling=True,
        validation_samples=128,
        validation_seed=123,
        lookback=5,
        verbose=False)

    # Print performance
    print('Best val loss = {:.8f}'.format(min(fastshap.loss_list)))

    # Save model
    modifier = 'paired_samples={}'.format(n_samples)
    explainer.cpu()
    torch.save(explainer, '../models/news_explainer {} nopenalty.pt'.format(modifier))

Best val loss = 0.00270184
Best val loss = 0.00032815
Best val loss = 0.00032703
Best val loss = 0.00032261
Best val loss = 0.00032042
Best val loss = 0.00032073


# Test other parameters

In [9]:
# Set up explainer model
explainer = nn.Sequential(
    nn.Linear(num_features, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 2 * num_features)).to(device)

# Set up FastSHAP wrapper
fastshap = FastSHAP(explainer, surr, normalization='additive', link=nn.Softmax(dim=1))

# Train
fastshap.train(
    X_train,
    X_val[:500],
    batch_size=32,
    num_samples=32,
    max_epochs=200,
    eff_lambda=0,
    paired_sampling=True,
    validation_samples=128,
    validation_seed=123,
    lookback=5,
    verbose=False)

# Print performance
print('Best val loss = {:.8f}'.format(min(fastshap.loss_list)))

# Save model
modifier = 'nopenalty'
explainer.cpu()
torch.save(explainer, '../models/news_explainer {}.pt'.format(modifier))

Best val loss = 0.00032219


In [10]:
# Set up explainer model
explainer = nn.Sequential(
    nn.Linear(num_features, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 2 * num_features)).to(device)

# Set up FastSHAP wrapper
fastshap = FastSHAP(explainer, surr, normalization=None, link=nn.Softmax(dim=1))

# Train
fastshap.train(
    X_train,
    X_val[:500],
    batch_size=32,
    num_samples=32,
    max_epochs=200,
    eff_lambda=0.1,
    paired_sampling=True,
    validation_samples=128,
    validation_seed=123,
    lookback=5,
    verbose=False)

# Print performance
print('Best val loss = {:.8f}'.format(min(fastshap.loss_list)))

# Save model
modifier = 'nonormalization'
explainer.cpu()
torch.save(explainer, '../models/news_explainer {}.pt'.format(modifier))

Best val loss = 0.00031003


In [11]:
# Set up explainer model
explainer = nn.Sequential(
    nn.Linear(num_features, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 2 * num_features)).to(device)

# Set up FastSHAP wrapper
fastshap = FastSHAP(explainer, surr, normalization=None, link=nn.Softmax(dim=1))

# Train
fastshap.train(
    X_train,
    X_val[:500],
    batch_size=32,
    num_samples=32,
    max_epochs=200,
    eff_lambda=0,
    paired_sampling=True,
    validation_samples=128,
    validation_seed=123,
    lookback=5,
    verbose=False)

# Print performance
print('Best val loss = {:.8f}'.format(min(fastshap.loss_list)))

# Save model
modifier = 'nopenalty nonormalization'
explainer.cpu()
torch.save(explainer, '../models/news_explainer {}.pt'.format(modifier))

Best val loss = 0.00030896
